# RADICAL-Cybertools: RADICAL-EnTK Pilot

One has to handle RADICAL-Pilot applications with some care when running them in a Jupyter notebook.  In particular one should avoid to run cells out of order.  It is usually best to cleanly terminate the kernel before rerunning any / all cells.  This notebook thus puts the exercise code into a *single* cell which you can edit freely and then execute.

## Exercise 1: Change the workload size and composition
  - submit more than one task type
    - change number of tasks
    - change time to sleep
    - change executable
    - change number or cores per rank
  - use the [RP documentation(https://radicalpilot.readthedocs.io/en/stable/tutorials/describing_tasks.html) for reference


In [ ]:
%env RADICAL_REPORT=True
%env RADICAL_REPORT_ANIME=False

import radical.pilot as rp
import radical.utils as ru

report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)

session = rp.Session()
pmgr    = rp.PilotManager(session=session)
tmgr    = rp.TaskManager(session=session)


report.header('submit pilot')

pd_init = {'resource'     : 'local.localhost',
           'runtime'      : 30,  # pilot runtime minutes
           'project'      : None,
           'queue'        : None,
           'cores'        : 4,
           'gpus'         : 0,
           'exit_on_error': True}
pdesc = rp.PilotDescription(pd_init)
pilot = pmgr.submit_pilots(pdesc)
tmgr.add_pilots(pilot)

report.header('execute workload')

import os
import random

n = 10

report.progress_tgt(n, label='create')
tds = list()
for i in range(n):

    td = rp.TaskDescription()
    td.executable     = 'radical-pilot-hello.sh'
    td.arguments      = [10]
    td.ranks          = 1
    td.cores_per_rank = 1

    tds.append(td)
    report.progress()

report.progress_done()

tasks = tmgr.submit_tasks(tds)
tmgr.wait_tasks()

report.header('inspect results')
for task in tasks:
    print('  * %s: %s' % (task.uid, task.state))

report.header('finalize')
session.close(cleanup=True)

In [ ]:
! ls $HOME/radical.pilot.sandbox/rp.session.*/pilot*/task.000000/

In [ ]:
! cat `ls $HOME/radical.pilot.sandbox/rp.session.*/pilot*/task.000000/task.000000.out`